# Per Capita By State

This visualization shows the top per capita confirmed cases of COVID-19 in US states.

In [2]:
import pandas as pd
import requests
import json
import numpy as np

def download_covidtracking():
    URL = 'https://covidtracking.com/api/states/daily'
    r = requests.get(URL)
    sd = pd.read_json(json.dumps(r.json()))
    sd['date'] = pd.to_datetime(sd['date'], format='%Y%m%d')
    return sd

def download_FIPS():
    return pd.read_csv('https://raw.githubusercontent.com/dsoto/simple-covid-models/master/state-populations.csv',
                        index_col='USPS')


def create_sorted_data(select_date):
    latest_covid_data = covid_data[covid_data['date']==select_date]
    # facilitate merge with population data
    latest_covid_data = latest_covid_data.set_index('state')
    # download state populations
    population_data = download_FIPS()
    # compute positive cases per capita 
    latest_covid_data['positive_per_capita'] = latest_covid_data['positive'] / population_data['population'] * 1E6
    # put data in decending order
    sorted_latest_covid_data = latest_covid_data.sort_values('positive_per_capita', ascending=False)
    return sorted_latest_covid_data

In [3]:
covid_data = download_covidtracking()
# filter to relevant columns
covid_data = covid_data[['date', 'positive', 'state']]
# choose earliest date in set
select_date = covid_data['date'].min()
# filter for single date
sorted_latest_covid_data = create_sorted_data(select_date)


In [4]:
# bqplot bar graph

def update_plot(*args):
    select_date = covid_data['date'].min() + np.timedelta64(date_slider.value, 'D')
    sorted_latest_covid_data = create_sorted_data(select_date)
    bar.x = sorted_latest_covid_data['positive_per_capita'][:10].index
    bar.y = sorted_latest_covid_data['positive_per_capita'][:10]
    date.value = str(select_date) 

from bqplot import (OrdinalScale, LinearScale, Bars, Label,
                    Figure, Axis, ColorScale, ColorAxis, CATEGORY10)
import ipywidgets as widgets

date_slider = widgets.IntSlider(value=0, min=0, max=30, step=1, description='Days:')
date_slider.observe(update_plot, 'value')
play_button = widgets.Play(min=0, max=24, interval=2000)
date = widgets.Text(str(covid_data['date'].min()))

widgets.jslink((play_button, 'value'), (date_slider, 'value'))


x_ord = OrdinalScale()
y_sc = LinearScale()

x = sorted_latest_covid_data['positive_per_capita'][:10].index
y = sorted_latest_covid_data['positive_per_capita'][:10]

bar = Bars(x=x, y=y, scales={'x': x_ord, 'y': y_sc},
           orientation='horizontal')
ax_x = Axis(scale=x_ord, orientation='vertical')
ax_y = Axis(scale=y_sc, tick_format='0.0f')

fig = Figure(marks=[bar], axes=[ax_x, ax_y], 
             padding_x=0.025, padding_y=0.025,
             animation_duration=1500,
             title='Positive Cases per 1 Million People')
box = widgets.VBox([fig, date, date_slider, play_button])
display(box)